<a href="https://colab.research.google.com/github/ananyabadkar/movie-ratings-Spark-practice-notebooks-/blob/main/notebooks/%20ML_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark -q

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import RFormula
import pandas as pd
import click


In [5]:
spark = SparkSession.builder.appName("App").getOrCreate()

In [16]:
from google.colab import files
uploaded = files.upload()


Saving airbnb_switzerland.csv to airbnb_switzerland.csv


In [17]:
filepath = "/content/airbnb_switzerland.csv"
airbnbDF = spark.read.csv(filepath, header=True, inferSchema=True)
airbnbDF.show(5)


+---+--------------------+-------+-------------+---------------+-----+--------------+-----------------+-----------+-----------------+----------------+
| id|                name|host_id|neighbourhood|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|availability_365|
+---+--------------------+-------+-------------+---------------+-----+--------------+-----------------+-----------+-----------------+----------------+
|101|Cozy Studio Zuric...|   2001|       Zurich|Entire home/apt|  120|             2|               45| 2025-08-10|              0.9|             300|
|102|Lake View Apartme...|   2002|       Geneva|Entire home/apt|  200|             3|               80| 2025-08-01|              1.2|             250|
|103|Budget Room Lausanne|   2003|     Lausanne|   Private room|   65|             1|               22| 2025-07-20|              0.5|             180|
|104|Luxury Penthouse ...|   2004|       Zurich|Entire home/apt|  350|             2|         

In [18]:
trainDF, testDF = airbnbDF.randomSplit([0.5, 0.2], seed=42)

print(f"""There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set.""")


There are 9 rows in the training set, and 6 in the test set.


In [29]:
vecAssembler = VectorAssembler(inputCols=[ "minimum_nights"], outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF)
vecTrainDF.select("minimum_nights","features", "price").show(5)



+--------------+--------+-----+
|minimum_nights|features|price|
+--------------+--------+-----+
|             2|   [2.0]|  120|
|             3|   [3.0]|  200|
|             2|   [2.0]|  350|
|             4|   [4.0]|  220|
|             1|   [1.0]|   70|
+--------------+--------+-----+
only showing top 5 rows



In [30]:
lr = LinearRegression(featuresCol="features", labelCol="price")
lrModel = lr.fit(vecTrainDF)


In [31]:
m = round(lrModel.coefficients[0],2)
b = round(lrModel.intercept,2)

print(f"""The formula for the linear regression line is price = {m}*minimum_nights + {b}.""")

The formula for the linear regression line is price = 66.38*minimum_nights + 14.87.


In [33]:
pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(trainDF)
lrPredictions = pipelineModel.transform(testDF)
lrPredictions.select("prediction","price","features").show(10)

+------------------+-----+--------+
|        prediction|price|features|
+------------------+-----+--------+
| 81.24999999999984|   65|   [1.0]|
|           147.625|  150|   [2.0]|
|           147.625|  180|   [2.0]|
| 346.7500000000004|  400|   [5.0]|
|           147.625|   85|   [2.0]|
|479.50000000000074|  500|   [7.0]|
+------------------+-----+--------+

